# Dataset preprocessing

In this notebook we prepare the dataset by preprocessing it through a SparkJob.

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql.types import FloatType, IntegerType

from pyspark.sql import *
from pyspark.sql import functions as sf
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date

In [2]:
sc = pyspark.SparkContext(appName="Preprocessing dataset")
spark = SparkSession(sc)

25/08/31 11:53:15 WARN Utils: Your hostname, dsbda-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/08/31 11:53:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/31 11:53:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Loading the dataset

In this section we load the dataset inside a Spark DataFrame.

In [3]:
hdfs_path = "hdfs://localhost:54310/user/ubuntu/dataset/"
schema = "BookID string, Title string, Authors string, Avg_Rating string, ISBN string, ISBN13 string, Language_Code string, Num_Pages string, Ratings_Count string, Text_Reviews_Count string, Publication_Date string, Publisher string"

df = spark.read.csv(hdfs_path, header=False, inferSchema=False, sep=",", quote='', escape='', schema=schema)

In [4]:
# Dropping uninformative columns
df=df.drop("BookID", "ISBN", "ISBN13", "Language_Code")

During the injection phase some escaping characters have been added. In particular here we aim to remove the _\\_, _[_, _]_ and _"_ characters.
In addition to that, we also removed the _'_ character only at the start and at the end of the row, in order to avoid dropping saxon genitives.

In [5]:
# Preprocessing
df = df.select([sf.regexp_replace(c, r'\\|\[|\]|\"', '').alias(c) for c in df.columns])
df = df.select([sf.regexp_replace(c, r'^ \'|\'$|', '').alias(c) for c in df.columns])

The next step is to cast each column to the appropriate type.

In [6]:
# Casting column types
df = df.withColumn("Avg_Rating", col("Avg_Rating").cast(FloatType())).withColumn("Num_Pages", col("Num_Pages").cast(IntegerType())).withColumn("Ratings_Count", col("Ratings_Count").cast(IntegerType())).withColumn("Text_Reviews_Count", col("Text_Reviews_Count").cast(IntegerType()))

# We have different formats for the date, so we unify them
df = df.withColumn("Pub_Date",
  sf.coalesce(
    to_date(col("Publication_Date"), "M/d/yyyy"),
    to_date(col("Publication_Date"), "MM/d/yyyy"),
    to_date(col("Publication_Date"), "M/dd/yyyy"),
    to_date(col("Publication_Date"), "MM/dd/yyyy"),
  ).alias("Pub_Date"),
)

df = df.drop("Publication_Date")

Finally, the preprocessed dataset is saved in Hadoop HDFS.

In [8]:
# Saving Preprocessed dataset on HDFS
hdfs_path_out = "hdfs://localhost:54310/user/ubuntu/dataset_preprocessed/"
df.write.csv(hdfs_path_out, header=False)

In [9]:
spark.stop()